# Import Libraries

In [ ]:
import ezc3d
import nimblephysics as nimble
import numpy as np
import os
import pandas as pd
from scipy.io import loadmat
from tqdm import tqdm

# Load AddBiomechanics B3D Reference File

In [36]:
# Load B3D file (AddBiomechanics Dataset)
b3d_file = "/home/mnt/Datasets/AddBiomechanics/test/With_Arm/Uhlrich2023_Formatted_With_Arm/subject2/subject2.b3d"
b3d_data = nimble.biomechanics.SubjectOnDisk(b3d_file)
b3d_frame = b3d_data.readFrames(0, 1)[0]
b3d_labels = [name for name, _ in b3d_frame.markerObservations]
b3d_points = np.array([obs[1] for obs in b3d_frame.markerObservations])

# Load 183 Subject MAT file

In [38]:
# Select subject and trial
ppid = "38"
trial_idx = 0

# Load joint centers for trial
jc_file = f"/home/mnt/Datasets/183_subjects/Kinematic_Data/{ppid}/JC_{ppid}.mat"
jc_data = loadmat(jc_file)
jc_vars = [k for k in jc_data.keys() if not k.startswith('__') and k != 'FILE_NAME']
for var in jc_vars:
    joint_center = jc_data[var][trial_idx]  # shape: (n_frames, 3)

# Load joint angles
ja_file = f"/home/mnt/Datasets/183_subjects/Kinematic_Data/{ppid}/JA_{ppid}.mat"
ja_data = loadmat(ja_file)
ja_vars = [k for k in ja_data.keys() if not k.startswith('__') and k != 'FILE_NAME']
for var in ja_vars:
    joint_angle = ja_data[var][trial_idx]  # shape: (n_frames, 3)

# Get trial file names
file_names = [str(f[0]) for f in jc_data['FILE_NAME'].squeeze()]

# Load subject info from Excel
info_file = "/home/mnt/Datasets/183_subjects/Participants Info/Subject Log.xlsx"
info_df = pd.read_excel(info_file)
subject_row = info_df[info_df['Subject ID'].astype(str) == str(ppid)]
gender = subject_row.iloc[0]['Gender']
age = subject_row.iloc[0]['Age']
height = subject_row.iloc[0]['Height (cm)']
mass = subject_row.iloc[0]['Mass (kg)']

In [ ]:

# Load joint angles
ja_file = f"/home/mnt/Datasets/183_subjects/Kinematic_Data/{ppid}/JA_{ppid}.mat"
ja_data = loadmat(ja_file)
ja_vars = [k for k in ja_data.keys() if not k.startswith('__') and k != 'FILE_NAME']
for var in ja_vars:
    joint_angle = ja_data[var][trial_idx]  # shape: (n_frames, 3)



Subject 38: Gender=Male, Age=24.6, Height=195.58, Mass=104.3


In [34]:
print(info_df.columns)

Index(['Subject ID', 'Gender', 'Age', 'Height (cm)', 'Mass (kg)', 'Level',
       'Sport', 'Have you had any Injuries in the past 10 years?',
       'How many Injuries have you had?', 'Injury 1', 'Injury 2', 'Injury 3',
       'Injury 4', 'Injury 5'],
      dtype='object')


In [26]:
print("joint_centers_list shape:", np.array(joint_centers_list).shape)
print("joint_centers shape:", joint_centers.shape)

joint_centers_list shape: (36, 1)
joint_centers shape: (1, 36)


In [21]:
print("Trial dir:", dir(trial))

Trial dir: ['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'addPass', 'getBasicTrialType', 'getDetectedTrialFeatures', 'getForcePlates', 'getHasManualGRFAnnotation', 'getMarkerObservations', 'getMissingGRFReason', 'getName', 'getOriginalTrialEndFrame', 'getOriginalTrialEndTime', 'getOriginalTrialName', 'getOriginalTrialStartFrame', 'getOriginalTrialStartTime', 'getPasses', 'getSplitIndex', 'getTimestep', 'getTrialLength', 'setAccObservations', 'setBasicTrialType', 'setCustomValues', 'setDetectedTrialFeatures', 'setEmgObservations', 'setExoTorques', 'setForcePlates', 'setGyroObservations', 'setHasManualGRFAnnotation', 'setMarkerNamesGuessed', 'setMarkerObservations', 'setMissingGRFReason', 'setName', 'setOriginalTrialEndF

In [18]:
print("Header type:", type(header))
print("Header dir:", dir(header))

Header type: <class 'nimblephysics_libs._nimblephysics.biomechanics.SubjectOnDiskHeader'>
Header dir: ['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', 'addProcessingPass', 'addTrial', 'filterTrials', 'getProcessingPasses', 'getQuality', 'getTrials', 'recomputeColumnNames', 'setAgeYears', 'setBiologicalSex', 'setCustomValueNames', 'setGroundForceBodies', 'setHeightM', 'setHref', 'setMassKg', 'setNotes', 'setNumDofs', 'setNumJoints', 'setQuality', 'setSubjectTags', 'setTrials', 'trimToProcessingPasses']


In [3]:
# Load C3D file (183 Subjects Dataset)
c3d_file_path = "/home/mnt/Datasets/183_subjects/Kinematic_Data/743/Generated_C3D_files/0743_drop jump_01.c3d"
c3d_data = ezc3d.c3d(c3d_file_path)
c3d_frame = c3d_data['data']['points'][:, :, 0]
c3d_labels = c3d_data['parameters']['POINT']['LABELS']['value']

# Keep only the physical markers (first occuring)
seen = set()
unique_labels = []
unique_points = []
for label, point in zip(c3d_labels, c3d_frame.T):
    if label not in seen:
        unique_labels.append(label)
        unique_points.append(point)
        seen.add(label)
c3d_labels = unique_labels
c3d_points = np.array(unique_points)

# Print the labels:
print(f"{len(c3d_labels)} C3D Markers: {c3d_labels}")

65 C3D Markers: ['FHEA', 'BHEA', 'RHEA', 'LHEA', 'T2', 'T8', 'RCLA', 'LCLA', 'STER', 'RSCA', 'RSHO', 'RBIC', 'RLAT', 'RMED', 'RFOR', 'LSHO', 'LBIC', 'LLAT', 'LMED', 'LFOR', 'RASI', 'LASI', 'RPSI', 'LPSI', 'RTRO', 'RTHI', 'RSHA', 'RHEE', 'RTOE', 'LTRO', 'LTHI', 'LHEE', 'LTOE', 'VHEA', 'VCRO', 'VRSH', 'VREL', 'VRFO', 'VRWR', 'VLSH', 'VLEL', 'VLFO', 'VLWR', 'VCLA', 'VNEC', 'VLLU', 'VRNI', 'VLNI', 'VFPE', 'VBPE', 'VVER', 'VLAT', 'VPEL', 'VRCH', 'VLCH', 'VRHI', 'VRKN', 'VRAN', 'VLHI', 'VLKN', 'VLAN', 'VORI', 'VNEG', 'VPOS', 'VFAK']


In [ ]:



if len(c3d_labels) == len(set(c3d_labels)):
    print("No duplicates in B3D markers.")
else:
    print("Duplicates found in B3D markers!")

51 B3D Markers: ['C7', 'L.ASIS', 'L.PSIS', 'L_5meta', 'L_HJC', 'L_HJC_reg', 'L_Shoulder', 'L_Sternum', 'L_ankle', 'L_calc', 'L_elbow_lat', 'L_elbow_med', 'L_forearm', 'L_humerus', 'L_knee', 'L_sh2', 'L_sh3', 'L_shank_antsup', 'L_thigh1', 'L_thigh2', 'L_thigh3', 'L_thigh4', 'L_toe', 'L_wrist_radius', 'L_wrist_ulna', 'R_HJC', 'R_HJC_reg', 'R_Shoulder', 'R_Sternum', 'R_elbow_lat', 'R_elbow_med', 'R_forearm', 'R_humerus', 'R_wrist_radius', 'R_wrist_ulna', 'r.ASIS', 'r.PSIS', 'r_5meta', 'r_ankle', 'r_calc', 'r_knee', 'r_sh2', 'r_sh3', 'r_sh4', 'r_shank_antsup', 'r_thigh1', 'r_thigh2', 'r_thigh3', 'r_thigh4', 'r_thigh5', 'r_toe']
64 C3D Markers: ['FHEA', 'BHEA', 'RHEA', 'LHEA', 'T2', 'T8', 'RCLA', 'LCLA', 'STER', 'RSCA', 'RSHO', 'RBIC', 'RLAT', 'RMED', 'RFOR', 'LSHO', 'LBIC', 'LLAT', 'LMED', 'LFOR', 'RASI', 'LASI', 'RPSI', 'LPSI', 'RTRO', 'RTHI', 'RSHA', 'RHEE', 'RTOE', 'LTRO', 'LTHI', 'LHEE', 'LTOE', 'VHEA', 'VCRO', 'VRSH', 'VREL', 'VRFO', 'VRWR', 'VLSH', 'VLEL', 'VLFO', 'VLWR', 'VCLA', 'VN

In [ ]:



if len(c3d_labels) == len(set(c3d_labels)):
    print("No duplicates in B3D markers.")
else:
    print("Duplicates found in B3D markers!")

51 B3D Markers: ['C7', 'L.ASIS', 'L.PSIS', 'L_5meta', 'L_HJC', 'L_HJC_reg', 'L_Shoulder', 'L_Sternum', 'L_ankle', 'L_calc', 'L_elbow_lat', 'L_elbow_med', 'L_forearm', 'L_humerus', 'L_knee', 'L_sh2', 'L_sh3', 'L_shank_antsup', 'L_thigh1', 'L_thigh2', 'L_thigh3', 'L_thigh4', 'L_toe', 'L_wrist_radius', 'L_wrist_ulna', 'R_HJC', 'R_HJC_reg', 'R_Shoulder', 'R_Sternum', 'R_elbow_lat', 'R_elbow_med', 'R_forearm', 'R_humerus', 'R_wrist_radius', 'R_wrist_ulna', 'r.ASIS', 'r.PSIS', 'r_5meta', 'r_ankle', 'r_calc', 'r_knee', 'r_sh2', 'r_sh3', 'r_sh4', 'r_shank_antsup', 'r_thigh1', 'r_thigh2', 'r_thigh3', 'r_thigh4', 'r_thigh5', 'r_toe']
64 C3D Markers: ['FHEA', 'BHEA', 'RHEA', 'LHEA', 'T2', 'T8', 'RCLA', 'LCLA', 'STER', 'RSCA', 'RSHO', 'RBIC', 'RLAT', 'RMED', 'RFOR', 'LSHO', 'LBIC', 'LLAT', 'LMED', 'LFOR', 'RASI', 'LASI', 'RPSI', 'LPSI', 'RTRO', 'RTHI', 'RSHA', 'RHEE', 'RTOE', 'LTRO', 'LTHI', 'LHEE', 'LTOE', 'VHEA', 'VCRO', 'VRSH', 'VREL', 'VRFO', 'VRWR', 'VLSH', 'VLEL', 'VLFO', 'VLWR', 'VCLA', 'VN

In [6]:
from collections import defaultdict

# Get marker labels and first frame points
c3d_labels = list(c3d_data['parameters']['POINT']['LABELS']['value'])
c3d_frame = c3d_data['data']['points'][:, :, 0].T  # shape: (n_markers, 3)

# Find duplicates and compare their values
duplicates = defaultdict(list)
for idx, label in enumerate(c3d_labels):
    duplicates[label].append(idx)

for label, idxs in duplicates.items():
    if len(idxs) > 1:
        values = [c3d_frame[i] for i in idxs]
        all_equal = all(np.allclose(values[0], v, equal_nan=True) for v in values[1:])
        print(f"Marker '{label}' occurs {len(idxs)} times. All values equal? {all_equal}")
        if not all_equal:
            print(f"  Values: {values}")

Marker 'RLAT' occurs 4 times. All values equal? False
  Values: [array([-200.7660675 ,  540.90460205,  787.90808105,    1.        ]), array([-30.06690979, 483.97970581, 556.59350586,   1.        ]), array([312.26715088, 563.77996826, 542.84484863,   1.        ]), array([195.46452332, 550.69549561, 118.665802  ,   1.        ])]
Marker 'RMED' occurs 4 times. All values equal? False
  Values: [array([-188.98109436,  627.16162109,  764.0513916 ,    1.        ]), array([-90.73155212, 429.30871582, 554.74591064,   1.        ]), array([317.36831665, 688.58258057, 517.13238525,   1.        ]), array([205.32467651, 677.72015381, 121.45571899,   1.        ])]
Marker 'LLAT' occurs 4 times. All values equal? False
  Values: [array([-1.40114777e+02,  1.11120190e+03,  7.39015381e+02,  1.00000000e+00]), array([-4.74912977e+00,  1.07732007e+03,  4.99171112e+02,  1.00000000e+00]), array([-201.57388306,  901.1975708 ,   27.91419029,    1.        ]), array([-646.79638672,  942.72729492,   37.4344902 ,   

In [4]:
print("Available biomechanics classes:", [x for x in dir(nimble.biomechanics) if not x.startswith('_')])

Available biomechanics classes: ['Anthropometrics', 'BasicTrialType', 'BatchGaitInverseDynamics', 'Beam', 'BilevelFitResult', 'C3D', 'C3DLoader', 'ContactRegimeSection', 'CortexStreaming', 'DataQuality', 'DetectedTrialFeature', 'DynamicsFitProblemConfig', 'DynamicsFitter', 'DynamicsInitialization', 'ForcePlate', 'Frame', 'FrameList', 'FramePass', 'FramePassList', 'IKErrorReport', 'IMUFineTuneProblem', 'InitialMarkerFitParams', 'LabelledMarkers', 'LilypadSolver', 'LinkBeam', 'LinkBeamSearch', 'MarkerBeamSearch', 'MarkerFitter', 'MarkerFitterState', 'MarkerFixer', 'MarkerInitialization', 'MarkerLabeller', 'MarkerLabellerMock', 'MarkerMultiBeamSearch', 'MarkerTrace', 'MarkersErrorReport', 'MissingGRFReason', 'MissingGRFStatus', 'MultiBeam', 'NeuralMarkerLabeller', 'OpenSimFile', 'OpenSimMocoTrajectory', 'OpenSimMot', 'OpenSimParser', 'OpenSimScaleAndMarkerOffsets', 'OpenSimTRC', 'ProcessingPassType', 'ResidualForceHelper', 'SkeletonConverter', 'StreamingIK', 'StreamingMarkerTraces', 'Stre

In [5]:
marker_map = {
    # Head markers
    'FHEA': None,  # Front head - no direct equivalent in B3D
    'BHEA': None,  # Back head - no direct equivalent
    'RHEA': None,  # Right head - no direct equivalent
    'LHEA': None,  # Left head - no direct equivalent
    'VHEA': None,  # Virtual head - no direct equivalent
    
    # Trunk markers
    'T2': 'C7',  # T2 vertebra is close to C7 anatomically
    'T8': None,  # No direct equivalent in B3D
    'STER': 'R_Sternum',  # Sternum
    'LCLA': 'L_Sternum',  # Left clavicle
    'RCLA': 'R_Sternum',  # Right clavicle
    'RSCA': 'R_Shoulder',  # Right scapula
    'VNEC': None,  # Virtual neck - no direct equivalent
    
    # Arm markers
    'RSHO': 'R_Shoulder',  # Right shoulder
    'RBIC': 'R_humerus',  # Right biceps
    'RMED': 'R_elbow_med',  # Right medial elbow
    'RLAT': 'R_elbow_lat',  # Right lateral elbow
    'RFOR': 'R_forearm',  # Right forearm
    'LSHO': 'L_Shoulder',  # Left shoulder
    'LBIC': 'L_humerus',  # Left biceps
    'LMED': 'L_elbow_med',  # Left medial elbow
    'LLAT': 'L_elbow_lat',  # Left lateral elbow
    'LFOR': 'L_forearm',  # Left forearm
    'VRWR': 'R_wrist_radius',  # Virtual right wrist
    'VLWR': 'L_wrist_ulna',  # Virtual left wrist
    
    # Pelvis markers
    'RASI': 'r.ASIS',  # Right ASIS
    'LASI': 'L.ASIS',  # Left ASIS
    'RPSI': 'r.PSIS',  # Right PSIS
    'LPSI': 'L.PSIS',  # Left PSIS
    'VPEL': None,  # Virtual pelvis - no direct equivalent
    
    # Right leg markers
    'RTRO': 'r_thigh1',  # Right trochanter
    'RTHI': 'r_thigh2',  # Right thigh
    'RSHA': 'r_shank_antsup',  # Right shank
    'RHEE': 'r_calc',  # Right heel
    'RTOE': 'r_toe',  # Right toe
    'VRKN': 'r_knee',  # Virtual right knee
    
    # Left leg markers
    'LTRO': 'L_thigh1',  # Left trochanter
    'LTHI': 'L_thigh2',  # Left thigh
    'LSHA': 'L_shank_antsup',  # Left shank
    'LHEE': 'L_calc',  # Left heel
    'LTOE': 'L_toe',  # Left toe
    'VLKN': 'L_knee',  # Virtual left knee
}

In [6]:
def similarity_transform(A, B):
    centroid_A = np.mean(A, axis=0)
    centroid_B = np.mean(B, axis=0)
    AA = A - centroid_A
    BB = B - centroid_B
    var_A = np.sum(AA ** 2)
    scale = np.sum(BB * AA) / var_A
    H = AA.T @ BB
    U, S, Vt = np.linalg.svd(H)
    R_mat = Vt.T @ U.T
    if np.linalg.det(R_mat) < 0:
        Vt[-1, :] *= -1
        R_mat = Vt.T @ U.T
    t = centroid_B - scale * R_mat @ centroid_A
    return scale, R_mat, t

In [7]:
# ...existing code...

# Build C3D label to index mapping
c3d_label_to_idx = {label: i for i, label in enumerate(c3d_markers)}

b3d_pts = []
c3d_pts = []
used_b3d_names = []
used_c3d_names = []

for c3d_marker, b3d_marker in marker_map.items():
    if b3d_marker and b3d_marker in b3d_markers and c3d_marker in c3d_label_to_idx:
        c3d_idx = c3d_label_to_idx[c3d_marker]
        b3d_idx = b3d_markers.index(b3d_marker)
        # Defensive: check bounds
        if c3d_idx < c3d_points.shape[0]:
            c3d_pts.append(c3d_points[c3d_idx])
            b3d_pts.append(frame.markerObservations[b3d_idx][1])
            used_c3d_names.append(c3d_marker)
            used_b3d_names.append(b3d_marker)

b3d_pts = np.array(b3d_pts)
c3d_pts = np.array(c3d_pts)

print("B3D markers used:", used_b3d_names)
print("C3D markers used:", used_c3d_names)
print("B3D points shape:", b3d_pts.shape)
print("C3D points shape:", c3d_pts.shape)
# ...existing code...

B3D markers used: ['C7', 'R_Sternum', 'L_Sternum', 'R_Sternum', 'R_Shoulder', 'R_Shoulder', 'R_humerus', 'R_elbow_med', 'R_elbow_lat', 'R_forearm', 'L_Shoulder', 'L_humerus', 'L_elbow_med', 'L_elbow_lat', 'L_forearm', 'R_wrist_radius', 'L_wrist_ulna', 'r.ASIS', 'L.ASIS', 'r.PSIS', 'L.PSIS', 'r_thigh1', 'r_thigh2', 'r_shank_antsup', 'r_calc', 'r_toe', 'r_knee', 'L_thigh1', 'L_thigh2', 'L_calc', 'L_toe', 'L_knee']
C3D markers used: ['T2', 'STER', 'LCLA', 'RCLA', 'RSCA', 'RSHO', 'RBIC', 'RMED', 'RLAT', 'RFOR', 'LSHO', 'LBIC', 'LMED', 'LLAT', 'LFOR', 'VRWR', 'VLWR', 'RASI', 'LASI', 'RPSI', 'LPSI', 'RTRO', 'RTHI', 'RSHA', 'RHEE', 'RTOE', 'VRKN', 'LTRO', 'LTHI', 'LHEE', 'LTOE', 'VLKN']
B3D points shape: (32, 3)
C3D points shape: (32, 3)


In [8]:
# ...existing code...

print("Any NaNs in c3d_pts?", np.isnan(c3d_pts).any())
print("Any NaNs in b3d_pts?", np.isnan(b3d_pts).any())
print("c3d_pts shape:", c3d_pts.shape)
print("b3d_pts shape:", b3d_pts.shape)

# Remove any pairs where either point is NaN
valid = ~np.isnan(c3d_pts).any(axis=1) & ~np.isnan(b3d_pts).any(axis=1)
c3d_pts_valid = c3d_pts[valid]
b3d_pts_valid = b3d_pts[valid]

print("Valid pairs:", c3d_pts_valid.shape[0])
print("Total pairs:", c3d_points.shape[0])

scale, R, t = similarity_transform(c3d_pts_valid, b3d_pts_valid)
print("Scale:", scale)
print("Rotation:\n", R)
print("Translation:", t)
# ...existing code...

Any NaNs in c3d_pts? True
Any NaNs in b3d_pts? False
c3d_pts shape: (32, 3)
b3d_pts shape: (32, 3)
Valid pairs: 28
Total pairs: 90
Scale: 0.5372631736370475
Rotation:
 [[ 0.59097651 -0.43563789  0.67894506]
 [ 0.35557516  0.89614574  0.26549788]
 [-0.72409466  0.08451298  0.68450309]]
Translation: [-0.61752315  0.33797044 -0.28958951]


In [ ]:
# ...existing code...

human_model = nimble.RajagopalHumanBodyModel()
print("RajagopalHumanBodyModel methods:", dir(human_model))
# ...existing code...

RajagopalHumanBodyModel methods: ['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'anatomicalMarkers', 'bodyScales', 'ignoredBodies', 'jointsDrivenBy', 'markersMap', 'meshMap', 'meshScaleMap', 'skeleton', 'trackingMarkers', 'warnings']
